# Tutorial 08 - Post-hoc Time Sync

Pupil Core distinguishes between `System Time` and `Pupil Time`, measured in seconds.

`System Time` is the current time of the device running Pupil Core software and uses the Unix epoc, while `Pupil Time` has an arbitrary that can be used to synchronize the clock between multiple devices.

Since the exported data (pupil, gaze, fixations, blinks, surface, etc.) uses timestamps in `Pupil Time`, it is often desireable to convert these timestamps into Unix timestamps (`System Time`), or into `datetime` objects in Python.

This tutorial shows how to easily perform the conversion and save the data in a new file.

---

> To execute this notebook, download the [sample recording](https://drive.google.com/file/d/1vzjZkjoi8kESw8lBnsa_k_8hXPf3fMMC/view?usp=sharing). Unzip and move it into the `recordings` directory for this repository.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!unzip /content/drive/MyDrive/sample_recording_v2.zip

Mounted at /content/drive
Archive:  /content/drive/MyDrive/sample_recording_v2.zip
   creating: sample_recording_v2/
  inflating: sample_recording_v2/eye1_timestamps.npy  
  inflating: sample_recording_v2/notify_timestamps.npy  
  inflating: sample_recording_v2/gaze_timestamps.npy  
  inflating: sample_recording_v2/pupil.pldata  
  inflating: sample_recording_v2/world.intrinsics  
  inflating: sample_recording_v2/eye0_lookup.npy  
  inflating: sample_recording_v2/world_timestamps.npy  
  inflating: sample_recording_v2/eye0.mp4  
  inflating: sample_recording_v2/eye0_timestamps.npy  
  inflating: sample_recording_v2/world.mp4  
  inflating: sample_recording_v2/square_marker_cache  
  inflating: sample_recording_v2/notify.pldata  
  inflating: sample_recording_v2/surface_definitions_v01  
  inflating: sample_recording_v2/eye1.mp4  
  inflating: sample_recording_v2/pupil_timestamps.npy  
  inflating: sample_recording_v2/surface_definitions  
  inflating: sample_recording_v2/eye1_lookup.np

In [2]:
import pathlib
import json

import numpy as np
import pandas as pd

pd.options.display.float_format = '{:}'.format

DATAFRAME_HEAD_COUNT = 3

First, we define the path to the recording directory, as well as the export directory within the recording.

In [3]:
# rec_dir = pathlib.Path(".").joinpath("recordings").joinpath("sample_recording_v2").absolute()
# assert rec_dir.is_dir(), "Please download the sample recording into 'recordings' directory."
rec_dir = "/content/sample_recording_v2"

In [12]:
# export_dir = rec_dir.joinpath("exports").joinpath("000")
# assert export_dir.is_dir(), "Please create at least one export."
export_dir = "/content/sample_recording_v2/exports/000"

The recording contains a meta-data file (`info.player.json`) which provide essential information about the recording itself, as well as the context in which it was made. More information about the format can be found [here](https://github.com/pupil-labs/pupil/blob/master/pupil_src/shared_modules/pupil_recording/README.md).

In [6]:
with pathlib.Path(rec_dir).joinpath("info.player.json").open() as file:
    meta_info = json.load(file)

meta_info

{'duration_s': 100.0,
 'meta_version': '2.2',
 'min_player_version': '2.0',
 'recording_name': '2019_10_24',
 'recording_software_name': 'Pupil Capture',
 'recording_software_version': '1.15.67',
 'recording_uuid': 'fbd7d03b-fd91-47c7-9c61-120c2af37779',
 'start_time_synced_s': 329353.31413932703,
 'start_time_system_s': 1571931006.836434,
 'system_info': 'User: mkassner, Platform: Darwin, Machine: moritzs-air.fritz.box, Release: 13.4.0, Version: Darwin Kernel Version 13.4.0: Mon Jan 11 18:17:34 PST 2016; root:xnu-2422.115.15~1/RELEASE_X86_64'}

Using the start time of the recording in `System Time` (`start_time_system_s` field) and in `Pupil Time` (`start_time_synced_s` field), we calculate the offset which will be applied to timestamps in other data files to convert them to Unix timestamps.

In [7]:
start_timestamp_unix = meta_info["start_time_system_s"]
start_timestamp_pupil = meta_info["start_time_synced_s"]
start_timestamp_diff = start_timestamp_unix - start_timestamp_pupil

## Pupil Positions Timestamps

The code bellow implements the following steps:
- Load the `pupil_positions.csv` file from the export directory into a Pandas dataframe
- Convert the `pupil_timestamp` column values to Unix timestamps (new `pupil_timestamp_unix` column)
- Convert the `pupil_timestamp` column values to datetime objects (new `pupil_timestamp_datetime` column)
- Save the updated dataframe into `pupil_positions_unix_datetime` file in the export directory

In [13]:
pupil_positions_df = pd.read_csv(pathlib.Path(export_dir).joinpath("pupil_positions.csv"))
pupil_positions_df.head(DATAFRAME_HEAD_COUNT)

,pupil_timestamp,world_index,eye_id,confidence,norm_pos_x,norm_pos_y,diameter,method,ellipse_center_x,ellipse_center_y,...,circle_3d_normal_y,circle_3d_normal_z,circle_3d_radius,theta,phi,projected_sphere_center_x,projected_sphere_center_y,projected_sphere_axis_a,projected_sphere_axis_b,projected_sphere_angle
0,329353.721339,0,1,0.9992698495763064,0.545727166321338,0.6300944755992544,50.19309954451988,3d c++,104.77961593369687,71.02186068494316,...,-0.1344528819807824,-0.7611991386996653,3.170051697723256,1.4359350164184437,-2.265608188206017,157.33537883128213,84.97669413085066,169.9658683385669,169.9658683385669,90.0
1,329353.722497,0,0,0.8821903959541288,0.4732410575900459,0.4165805522871153,47.71385406493233,3d c++,90.86228305728882,112.01653396087386,...,0.0631258415487939,-0.8160214865410084,3.249497262990368,1.633964168433327,-2.184271545863164,136.35058824450832,105.4072243147504,157.81606567180546,157.81606567180546,90.0
2,329353.726346,0,1,0.9990173937608026,0.5456551817541526,0.6301947804353969,50.32467239230429,3d c++,104.7657948967973,71.0026021564038,...,-0.1346691897581441,-0.7610185567039167,3.1784394686822828,1.435716723332044,-2.265857229927503,157.33537883128213,84.97669413085066,169.9658683385669,169.9658683385669,90.0


In [14]:
pupil_positions_df["pupil_timestamp_unix"] = pupil_positions_df["pupil_timestamp"] + start_timestamp_diff
pupil_positions_df.head(DATAFRAME_HEAD_COUNT)

,pupil_timestamp,world_index,eye_id,confidence,norm_pos_x,norm_pos_y,diameter,method,ellipse_center_x,ellipse_center_y,...,circle_3d_normal_z,circle_3d_radius,theta,phi,projected_sphere_center_x,projected_sphere_center_y,projected_sphere_axis_a,projected_sphere_axis_b,projected_sphere_angle,pupil_timestamp_unix
0,329353.721339,0,1,0.9992698495763064,0.545727166321338,0.6300944755992544,50.19309954451988,3d c++,104.77961593369687,71.02186068494316,...,-0.7611991386996653,3.170051697723256,1.4359350164184437,-2.265608188206017,157.33537883128213,84.97669413085066,169.9658683385669,169.9658683385669,90.0,1571931007.2436335
1,329353.722497,0,0,0.8821903959541288,0.4732410575900459,0.4165805522871153,47.71385406493233,3d c++,90.86228305728882,112.01653396087386,...,-0.8160214865410084,3.249497262990368,1.633964168433327,-2.184271545863164,136.35058824450832,105.4072243147504,157.81606567180546,157.81606567180546,90.0,1571931007.2447915
2,329353.726346,0,1,0.9990173937608026,0.5456551817541526,0.6301947804353969,50.32467239230429,3d c++,104.7657948967973,71.0026021564038,...,-0.7610185567039167,3.1784394686822828,1.435716723332044,-2.265857229927503,157.33537883128213,84.97669413085066,169.9658683385669,169.9658683385669,90.0,1571931007.2486405


In [15]:
pupil_positions_df["pupil_timestamp_datetime"] = pd.to_datetime(pupil_positions_df["pupil_timestamp_unix"], unit="s")
pupil_positions_df.head(DATAFRAME_HEAD_COUNT)

,pupil_timestamp,world_index,eye_id,confidence,norm_pos_x,norm_pos_y,diameter,method,ellipse_center_x,ellipse_center_y,...,circle_3d_radius,theta,phi,projected_sphere_center_x,projected_sphere_center_y,projected_sphere_axis_a,projected_sphere_axis_b,projected_sphere_angle,pupil_timestamp_unix,pupil_timestamp_datetime
0,329353.721339,0,1,0.9992698495763064,0.545727166321338,0.6300944755992544,50.19309954451988,3d c++,104.77961593369687,71.02186068494316,...,3.170051697723256,1.4359350164184437,-2.265608188206017,157.33537883128213,84.97669413085066,169.9658683385669,169.9658683385669,90.0,1571931007.2436335,2019-10-24 15:30:07.243633509
1,329353.722497,0,0,0.8821903959541288,0.4732410575900459,0.4165805522871153,47.71385406493233,3d c++,90.86228305728882,112.01653396087386,...,3.249497262990368,1.633964168433327,-2.184271545863164,136.35058824450832,105.4072243147504,157.81606567180546,157.81606567180546,90.0,1571931007.2447915,2019-10-24 15:30:07.244791508
2,329353.726346,0,1,0.9990173937608026,0.5456551817541526,0.6301947804353969,50.32467239230429,3d c++,104.7657948967973,71.0026021564038,...,3.1784394686822828,1.435716723332044,-2.265857229927503,157.33537883128213,84.97669413085066,169.9658683385669,169.9658683385669,90.0,1571931007.2486405,2019-10-24 15:30:07.248640537


In [17]:
pupil_positions_df.to_csv(pathlib.Path(export_dir).joinpath("pupil_positions_unix_datetime.csv"))

Bellow, the same steps are used to convert and save Unix and datetime timestamps for gaze and fixation data

## Gaze Positions Timestamps

In [20]:
gaze_positions_df = pd.read_csv(pathlib.Path(export_dir).joinpath("gaze_positions.csv"))
gaze_positions_df["gaze_timestamp_unix"] = gaze_positions_df["gaze_timestamp"] + start_timestamp_diff
gaze_positions_df["gaze_timestamp_datetime"] = pd.to_datetime(gaze_positions_df["gaze_timestamp_unix"], unit="s")
gaze_positions_df.to_csv(pathlib.Path(export_dir).joinpath("gaze_positions_unix_datetime.csv"))
gaze_positions_df.head(DATAFRAME_HEAD_COUNT)

,gaze_timestamp,world_index,confidence,norm_pos_x,norm_pos_y,base_data,gaze_point_3d_x,gaze_point_3d_y,gaze_point_3d_z,eye_center0_3d_x,...,gaze_normal0_y,gaze_normal0_z,eye_center1_3d_x,eye_center1_3d_y,eye_center1_3d_z,gaze_normal1_x,gaze_normal1_y,gaze_normal1_z,gaze_timestamp_unix,gaze_timestamp_datetime
0,329353.721918,0,0.8821903959541288,0.4669896617504211,0.5883357161056366,329353.722497-0 329353.721339-1,-11.80305702217247,-15.08821853022653,156.32943414013772,17.945315826323466,...,-0.1931468158806391,0.9679276177449296,-39.437908161701145,15.124464001056335,-21.51759382870768,0.1517262317211633,-0.133904369159789,0.9793103545493712,1571931007.2442126,2019-10-24 15:30:07.244212627
1,329353.7244215,0,0.8821903959541288,0.4670338939421431,0.58843611171472,329353.722497-0 329353.726346-1,-11.782673101225898,-15.090093055973252,156.20117441762542,17.945315826323466,...,-0.1931468158806391,0.9679276177449296,-39.437908161701145,15.124464001056335,-21.51759382870768,0.1519381672896685,-0.1341477922953073,0.9792441795297802,1571931007.246716,2019-10-24 15:30:07.246716022
2,329353.7269245,0,0.9987012967388026,0.4664144365037053,0.5887387216742821,329353.727503-0 329353.726346-1,-11.88373975977868,-15.070357239116316,155.5415153099811,17.945315826323466,...,-0.1940795624387817,0.9675583224652202,-39.437908161701145,15.124464001056335,-21.51759382870768,0.1519381672896685,-0.1341477922953073,0.9792441795297802,1571931007.249219,2019-10-24 15:30:07.249218941


## Fixations Timestamps

In [22]:
fixations_df = pd.read_csv(pathlib.Path(export_dir).joinpath("fixations.csv"))
fixations_df["start_timestamp_unix"] = fixations_df["start_timestamp"] + start_timestamp_diff
fixations_df["start_timestamp_datetime"] = pd.to_datetime(fixations_df["start_timestamp_unix"], unit="s")
fixations_df.to_csv(pathlib.Path(export_dir).joinpath("fixations_unix_datetime.csv"))
fixations_df.head(DATAFRAME_HEAD_COUNT)

,id,start_timestamp,duration,start_frame_index,end_frame_index,norm_pos_x,norm_pos_y,dispersion,confidence,method,gaze_point_3d_x,gaze_point_3d_y,gaze_point_3d_z,base_data,start_timestamp_unix,start_timestamp_datetime
0,2,329353.6292975,217.78349997475743,0,1,0.4642323025672498,0.5880829589443063,1.2388615445887934,0.9678853680864972,3d gaze,-12.347013257806694,-14.90324011940868,154.76171455502785,329353.6292975 329353.6318005 329353.6343035 3...,1571931007.151592,2019-10-24 15:30:07.151592016
1,3,329353.8495845,142.6859999774024,1,3,0.4522013890961779,0.6008539011713447,1.0018469124779923,0.989386261287357,3d gaze,-15.282994775465674,-16.74477203268101,154.72124511514812,329353.8495845 329353.85208750004 329353.85459...,1571931007.371879,2019-10-24 15:30:07.371879101
2,4,329354.2400935,217.78399997856468,10,17,0.4499159268627787,0.6185209769939535,1.4132714085849043,0.9451362525142144,3d gaze,-16.638742114802923,-20.200096359756834,162.2076645605484,329354.2400935 329354.2451 329354.247603 32935...,1571931007.762388,2019-10-24 15:30:07.762387991


## Surfaces Timestamps

In [23]:
surfaces_dir = pathlib.Path(export_dir).joinpath("surfaces")
assert surfaces_dir.is_dir(), "Please add at least one surface to the export."
surfaces_dir

PosixPath('/content/sample_recording_v2/exports/000/surfaces')

To aid in converting multiple files, some of which have more than one column with timestamp values, the `convert_and_save_timestamps` function is defined bellow, which replicates the steps previously described.

In [24]:
def convert_and_save_timestamps(input_path, column_names, timestamp_offset=start_timestamp_diff):

    output_path = input_path.with_name(input_path.stem + "_unix_datetime").with_suffix(input_path.suffix)

    df = pd.read_csv(input_path)

    for column_name in column_names:
        unix_column_name = column_name + "_unix"
        datetime_column_name = column_name + "_datetime"

        df[unix_column_name] = df[column_name] + timestamp_offset
        df[datetime_column_name] = pd.to_datetime(df[unix_column_name], unit="s")

    df.to_csv(output_path)

    return df.head(DATAFRAME_HEAD_COUNT)

In [25]:
convert_and_save_timestamps(
    input_path=surfaces_dir.joinpath("surface_events.csv"),
    column_names=["world_timestamp"]
)

,world_index,world_timestamp,surface_name,event_type,world_timestamp_unix,world_timestamp_datetime
0,115,329357.774241,Cover,enter,1571931011.2965355,2019-10-24 15:30:11.296535492
1,129,329358.243387,Cover,exit,1571931011.7656815,2019-10-24 15:30:11.765681505
2,160,329359.282209,Cover,enter,1571931012.8045034,2019-10-24 15:30:12.804503441


In [26]:
convert_and_save_timestamps(
    input_path=surfaces_dir.joinpath("surf_positions_Cover.csv"),
    column_names=["world_timestamp"]
)

,world_index,world_timestamp,img_to_surf_trans,surf_to_img_trans,num_detected_markers,dist_img_to_surf_trans,surf_to_dist_img_trans,world_timestamp_unix,world_timestamp_datetime
0,115,329357.774241,[[ 2.63960561e-03 2.58024554e-04 -1.48712940e...,[[ 4.32782785e+02 2.29303603e+02 4.93747392e...,2,[[ 3.02851290e-03 1.94267257e-04 -1.67875971e...,[[ 3.65641730e+02 1.45952680e+02 5.10581853e...,1571931011.2965355,2019-10-24 15:30:11.296535492
1,116,329357.807752,[[ 2.61582559e-03 1.98508430e-04 -1.40122220e...,[[ 4.35723386e+02 2.11060864e+02 4.81434227e...,2,[[ 3.02976495e-03 1.12532247e-04 -1.59118345e...,[[ 3.59978569e+02 1.28609103e+02 4.99827144e...,1571931011.3300464,2019-10-24 15:30:11.330046415
2,117,329357.841262,[[ 2.61877390e-03 1.73288569e-04 -1.35359237e...,[[ 4.32540891e+02 1.99779003e+02 4.69328973e...,2,[[ 3.06237719e-03 7.00499775e-05 -1.54690935e...,[[ 3.49716350e+02 1.17193549e+02 4.89468255e...,1571931011.3635566,2019-10-24 15:30:11.363556623


In [27]:
convert_and_save_timestamps(
    input_path=surfaces_dir.joinpath("gaze_positions_on_surface_Cover.csv"),
    column_names=["world_timestamp", "gaze_timestamp"]
)

,world_timestamp,world_index,gaze_timestamp,x_norm,y_norm,x_scaled,y_scaled,on_surf,confidence,world_timestamp_unix,world_timestamp_datetime,gaze_timestamp_unix,gaze_timestamp_datetime
0,329357.774241,115,329357.757756,0.7460118532180786,0.4544361531734466,156.6624891757965,131.78648442029953,True,0.4502635028608438,1571931011.2965355,2019-10-24 15:30:11.296535492,1571931011.2800505,2019-10-24 15:30:11.280050516
1,329357.774241,115,329357.761605,0.2059563249349594,0.3370377719402313,43.25082823634148,97.74095386266708,True,0.8969345939486267,1571931011.2965355,2019-10-24 15:30:11.296535492,1571931011.2838995,2019-10-24 15:30:11.283899546
2,329357.774241,115,329357.762762,0.7173773050308228,0.4465508460998535,150.64923405647278,129.49974536895752,True,0.4921131527276227,1571931011.2965355,2019-10-24 15:30:11.296535492,1571931011.2850566,2019-10-24 15:30:11.285056591


In [28]:
convert_and_save_timestamps(
    input_path=surfaces_dir.joinpath("fixations_on_surface_Cover.csv"),
    column_names=["world_timestamp", "start_timestamp"]
)

,world_timestamp,world_index,fixation_id,start_timestamp,duration,dispersion,norm_pos_x,norm_pos_y,x_scaled,y_scaled,on_surf,world_timestamp_unix,world_timestamp_datetime,start_timestamp_unix,start_timestamp_datetime
0,329357.975304,121,18,329357.972458,80.1039999932982,1.48319849750944,0.8056994676589966,0.93309485912323,169.19688820838928,270.5975091457367,True,1571931011.4975984,2019-10-24 15:30:11.497598410,1571931011.4947524,2019-10-24 15:30:11.494752407
1,329358.008814,122,18,329357.972458,80.1039999932982,1.48319849750944,0.8356009125709534,0.9337056279182434,175.4761916399002,270.7746320962906,True,1571931011.5311086,2019-10-24 15:30:11.531108618,1571931011.4947524,2019-10-24 15:30:11.494752407
2,329358.042324,123,18,329357.972458,80.1039999932982,1.48319849750944,0.8600224256515503,0.9362438917160034,180.6047093868256,271.510728597641,True,1571931011.5646186,2019-10-24 15:30:11.564618587,1571931011.4947524,2019-10-24 15:30:11.494752407
